In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [27]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs

from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry_new, decoder_with_fc_only
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tf_lab.iclr.helper import load_multiple_version_of_pcs

from tf_lab.nips.helper import pclouds_with_zero_mean_in_unit_sphere

In [26]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
n_pc_samples = 2048
do_training = True
first_time_running = True
load_model = False
seed = 42
max_training_epochs = 2000
loss = 'chamfer'
z_rotate = True
experiment_name = 'no_conv_deeper_snc_rotated_zero_mean_pcs' + str(n_pc_samples) +  'pts_' + loss

In [20]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'

In [7]:
n_threads = 25
verbose = True
version = 'centered'
full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/', version, str(n_pc_samples))
full_file_names = [f for f in files_in_subdirs(full_pclouds_path, '.ply')]
pclouds, _, _ = load_point_clouds_from_filenames(full_file_names, n_threads, snc_loader, verbose)
train_data = PointCloudDataSet(pclouds)
print train_data.num_examples

/orions4-zfs/projects/optas/Git_Repos/tf_lab/point_clouds/in_out.py:38: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


57449 pclouds were loaded. They belong in 57 shape-classes.
57449


In [17]:
pclouds = pclouds_with_zero_mean_in_unit_sphere(pclouds)
train_data = PointCloudDataSet(pclouds)

In [21]:
train_dir = osp.join('/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc', experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/no_conv_deeper_snc_rotated_zero_mean_pcs2048pts_chamfer'

In [30]:
n_input = [n_pc_samples, 3]

encoder_args = {'n_filters': [128, 128, 256, 512],
                'filter_sizes': [1, 1, 1, 1],
                'strides': [1, 1, 1, 1],
                'b_norm': True
               }

decoder_args = {'layer_sizes': [512, 1024, np.prod(n_input)],
                'b_norm': True
               }

In [31]:
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    
    if conf.train_dir != train_dir: # added to address moving training-folder to other location
        conf.train_dir = train_dir
        conf.save(osp.join(conf.train_dir, 'configuration'))

else:    
    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = z_rotate,
                encoder = encoder_with_convs_and_symmetry_new,
                encoder_args = encoder_args,
                decoder = decoder_with_fc_only,
                decoder_args = decoder_args,            
               )
    conf.allow_gpu_growth = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

In [33]:
if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)
        
if load_model and do_training:
    conf.training_epochs -= last_epoch
    print conf.training_epochs

In [ ]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '1.9725', 'loss=', '0.007701469')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/no_conv_deeper_snc_rotated_zero_mean_pcs2048pts_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '2.0508', 'loss=', '0.002637646')
('Epoch:', '0003', 'training time (minutes)=', '2.0274', 'loss=', '0.002032196')
('Epoch:', '0004', 'training time (minutes)=', '2.0699', 'loss=', '0.001680620')
('Epoch:', '0005', 'training time (minutes)=', '2.0705', 'loss=', '0.001520876')
('Epoch:', '0006', 'training time (minutes)=', '2.0989', 'loss=', '0.001372157')
('Epoch:', '0007', 'training time (minutes)=', '2.1456', 'loss=', '0.001297102')
('Epoch:', '0008', 'training time (minutes)=', '2.1748', 'loss=', '0.001190408')
('Epoch:', '0009', 'training time (minutes)=', '2.0788', 'loss=', '0.001150427')
('Epoch:', '0010', 'training time (minutes)=', '2.0361', 'loss=', '